In [1]:
# !pip3 install numpy matplotlib autograd

In [2]:
def mae(params, x, y):
    f = F(x, D)
    y_pred = f@params
    return np.mean(np.abs(y-y_pred))

def mse(params, x, y):
    f = F(x, D)
    y_pred = f@params
    return np.mean((y - y_pred)**2)

# Regression

**Goal**: teach computer to predict a numerical value given some input

**Model**. Consider approach based on regression analyis.
[Regression analysis](https://en.wikipedia.org/wiki/Regression_analysis) is a statistical technique to estimate dependence between variables. 
Suppose we have samples $x_n$ of a random value $X$ and the corresponding samples $y_n$ of a random value $Y$.
We assume $Y$ is a function $f$ of $X$ plus an error term independent of $X$:

$$
Y=f(X;\theta)+W.
$$

The function $f$ defines possible relationship between the variables and the parameters $\theta$ controls exact form of the dependence.


**Example 1.** Implement in Python function to sample the function $y=\sin x$ with addition of normally distributed noise. 

In [3]:
# import numpy as np
import autograd as ag
import autograd.numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

In [4]:
S = 100 # sample size
X = np.random.rand(S)*2 # inicialise X array
# print(f'min X = {np.min(X)}, max X = {np.max(X)}') 

W = np.random.randn(S)*0.01 # initialize noise
Y = np.sin(X) + W # create Y samples as sin function with noise
plt.plot(X,Y, '.') # ploting samples
plt.xlabel('$X$')
plt.ylabel('$Y=f(X)+W$')
plt.show()

**Approaches**. In fact as far as we need to find best function that approximate input, we will solve an optimization problem. Thus we need to define something to optimize. Mainly two functions are used: **loss** and **likelihood** functions.

**Minimum loss function principle**.
Function $f$ is a solution controlled by the parameter $\theta$. The optimal $\theta$ provides the best fitting function. To find optimal value we need to solve optimization problem or find minimum of some error function. Define **Loss function**:

$$
R[x,y] = ||y - f(x,\theta)||^2
$$

The solution of optimisation problem is $\theta_{opt} = \mathrm{argmin}_\theta(L)$

In practice, the estimate of $\theta$ is obtained by the [least squares method](https://en.wikipedia.org/wiki/Least_squares)
from minimization of the mean square error:

$$
R[\theta] = \sum_n (y_n-f(x_n,\theta))^2.
$$

The minimum of $R$ can be found from the necessary condition of optimality, which is sufficient in the case of quadratic functional:

$$
\frac{\partial R}{\partial\theta_k} = 2\sum_n(y_n-f(x_n,\theta_k))\frac{\partial f}{\partial \theta_k}(x_n,\theta) = 0\forall k.
$$

**Example 2.** Implement a function that solve linear least squares method for decomposition of $y$ over polynomials of degree $D$
on the interval $[0,1]$
using Moore-Penrose inverse:

$$
f(x,\theta)=\sum_{k=0}^D \theta_k x^k.
$$ 


In [5]:
def f(x, theta):
  xn = np.power(x, np.arange(len(theta)))
  return np.dot(xn, theta)

# theta =        1, x, x^2,   x^3, x^4,       x^5
theta_optimal = np.array([0, 1,   0,-1/2/3,   0, 1/2/3/4/5])

x0 = 0.1 
print(x0, f(x0,theta_optimal), np.sin(x0))

0.1 0.09983341666666667 0.09983341664682815


In [6]:
def F(x, D): # basis
  """
  Arguments:
    D - number of parameters.
  """
  return np.power(x[:,None], np.arange(D)[None,:])

def f(x, theta):
  # axis 0 = # sample
  # axis 1 = monomial degree
  xn = F(x, D=len(theta))
  return np.dot(xn, theta)

y_optimal = f(X, theta_optimal)

plt.plot(X,Y, '.b', label='samples')
plt.plot(X,y_optimal, '.r', label='optimal')
plt.xlabel('$X$')
plt.ylabel('$Y=f(X)+W$')
plt.legend()
plt.show()

**Example 3**. Calculate mean square error for obtained solution.

In [7]:
def R(theta, x, y):
  rn = (y - f(x, theta))**2
  return np.sum(rn)

print('Mean square error', R(theta=theta_optimal, x=X, y=Y)/np.sqrt(S) )

Mean square error 0.0013711085269987216


**Maximum likelihood principle**. Another way to approach machine learning problem is a search of maximum of some likelihood function. Likelihood function function is a probability of specific outcome. 
Given probability density $f_W$ for random noise $W$, we can estimate likelihood of outcomes resulting in samples $(x_n,y_n)$.
[Likelihood function](https://en.wikipedia.org/wiki/Likelihood_function) for the continuous destribution can be written as follows:
$$
L(\theta|x,y) = \prod_n f_W(y_n-f(x_n,\theta)). 
$$

$$
\log L(\theta|x,y) = \sum_n \log f_W(y_n-f(x_n,\theta)). 
$$

Maximum of the likelihood function corresponds to the value of $\theta$ such that outcomes $(x_n,y_n)$ are most probable,
thus we obtain the maximum likelihood estimate for the parameters 

$$
\hat\theta = \mathrm{argmax}_\theta \log L(\theta|x,y).
$$

**Problem 1.** Which distribution of $W$ does correspond to least squares method (minimum of mean square error coincides with maximum of likelihood function)?


# Answer 1

Normal

**Training**.
From the point of view of machine learning (ML), the function $R$ is a loss function measuring performance of the artificial neural network (ANN) $f$ with parameters $\theta$. 
The network is used to predict the value of $Y$ given a value of $X$.  
In the context of ML the optimization of $\theta$ above is called training of the algorithm/network.

The simplest method to train an algorithm is the [steepest gradient method](https://en.wikipedia.org/wiki/Gradient_descent).
Given an initial vector of parameters $\theta$, the parameters are updated on each step of optimization by the formula:

$$
\theta\mapsto \theta-\alpha\frac{\partial R}{\partial\theta}[\theta],
$$
until the method converges, i.e. the stop conditions are satisfied, e.g. the loss $R$ is small enough,
or its update $\partial R/\partial \theta$ is small.
The parameter $\alpha$ is learning rate in ML or step size in the numerical methods, it is a metaparameter, i.e. it does not belong to the model, but may affect the result of optimization. 
In the simplest form of the gradient descend the learning rate is constant, however often sofisticated method of variation of $\alpha$ are required 
to stabilze the method or improve convergence, e.g. [Adam method](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam).
Higher order method, such as [LBFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm), can also be used to improve convergence, however the methods are used less often.

The number of samples sometimes much larger than required to estimate the direction of optimization reliably,
in the case the extra points only slow down the computation of the gradient.
Therefore [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) is commonly used in ML instead of the gradient method.
In the method only a subset (batch) of the samples is used to compute the gradient of loss,
the batch is regenerated/resampled after few iterations of optimization. 


The most well-known variant of the least square method is the linear one/linear regression.
In the case prediction function $f$ depends on $\theta$ linearly:

$$
f(x,\theta)=\sum_k f_k(x)\theta_k,
$$
where $f_k$ is a set of predefined functions forming a basis (for instance, in **Example 2** we used the basis of polynomials).
Then optimality conditions takes form of a linear system with respect to $\theta$:

$$
\frac{\partial R}{\partial\theta_k} = 2\sum_n\bigg(y_n-\sum_j f_j(x_n)\theta_j\bigg)f_k(x_n) = 0\forall k.
$$

$$
F^T y - F^T F \theta = 0,
$$
where $F$ is a matrix with elements $F_{nk}=f_k(x_n)$

$$
F^T F \theta=F^T y.
$$
The solution to the problem is given by the [Moore-Penrose inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) of the matrix $F$:

$$
\hat \theta=(F^T F)^{-1}(F^T y).
$$ 
Numerically the solution can be computed without computation the square of the matrix $F$ using e.g. [QR-decomposition](https://en.wikipedia.org/wiki/Numerical_methods_for_linear_least_squares).

Unfortunatelly, it is hard to solve least square problem explicitly for dependencies $f$ more complex than linear. 
Hopefully, ML methods can be applied to arbitrary sufficiently smooth functions $f$.

**Example 4**. Implement function to solve least square problem and compere obtained parameter value with optimal one.

In [8]:
def lsq(x, y, D):
  f = F(x, D)
  theta = np.linalg.solve( f.T@f, f.T@y )
  return theta

D = len(theta_optimal)
theta_lsq = lsq(X, Y, D)
print(f'opt: {theta_optimal}')
print(f'lsq: {theta_lsq}')


opt: [ 0.          1.          0.         -0.16666667  0.          0.00833333]
lsq: [-8.87891989e-04  1.00594158e+00  1.71161078e-02 -2.02543224e-01
  2.28612940e-02  2.63477766e-03]


**Problem 2.** Split the data set $(x_n,y_n)$ to the training set (80%) and testing set (20%) and choose optimal $D$ avoiding overfitting.

# Answer 2

In [9]:
import matplotlib.pyplot as plt
from cycler import cycler
def plot_settings():
    plt.rcParams["axes.facecolor"] = '#0d1117'
    plt.rcParams["figure.facecolor"] = '#0d1117'

    # plt.rcParams['figure.figsize'] = [7.0, 3.0]
    plt.rcParams['figure.dpi'] = 100

    plt.rcParams["legend.labelcolor"] = 'w'
    plt.rcParams["axes.titlecolor"] = "w"

    # plt.rcParams["axes.spines.bottom.color"]
    # plt.rcParams["axes.spines.left"] = '#0d1117'
    plt.rcParams["axes.spines.right"] = False
    plt.rcParams["axes.spines.top"] = False

    plt.rcParams["axes.edgecolor"] = "#eef7f4"

    plt.rcParams["xtick.color"] = '#eef7f4'
    plt.rcParams["ytick.color"] = '#eef7f4'


    plt.rcParams["axes.labelcolor"] = '#eef7f4'

    plt.rcParams["grid.color"] = '#eef7f4'

    plt.rcParams["legend.frameon"] = False

    plt.rcParams['axes.prop_cycle'] = cycler(color=['g', 'r', 'b', 'y'])

plot_settings()

In [10]:
lsq_params

NameError: name 'lsq_params' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"train length: {len(y_train)}\ntest length: {len(y_test)}")
Ds = np.arange(1, 100, dtype=np.float64)
test_loss = np.empty_like(Ds)
train_loss = np.empty_like(Ds)
# plt.scatter(X_test, y_test)
min_loss = float("inf")
for i, D in enumerate(Ds):
    params = lsq(X_train, y_train, D)
    test_loss[i] = mean_squared_error(y_test, F(X_test, D)@params)
    train_loss[i] = mean_squared_error(y_train, F(X_train, D)@params)
    if test_loss[i] < min_loss:
        min_loss = test_loss[i]
        lsq_params = params
    # test_loss[i] = R(params, X_test, y_test)
    # train_loss[i] = R(params, X_train, y_train)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.scatter(Ds[:20], test_loss[:20],label="test loss")
ax1.scatter(Ds[:20], train_loss[:20],label="train loss")
ax1.set_yscale("log")
ax1.set_xlabel("D")
ax1.set_ylabel("MSE")
ax1.legend()

ax2.scatter(Ds, test_loss,label="test loss")
ax2.scatter(Ds, train_loss,label="train loss")
ax2.set_yscale("log")
ax2.set_xlabel("D")
ax2.set_ylabel("MSE")
ax2.legend()


In [ ]:
print("train loss:", *train_loss[:7])
print("test  loss:",*test_loss[:7])


In the examples above we used explicit results of differentiation but in practice gradient calculation performed by computer. For example, [``autograd``](https://github.com/HIPS/autograd) package is a tool to automatically differentiate native Python and Numpy code.

**Example 5** Create data array, define function and calculate its gradient using autograd

In [ ]:
# import autograd as ag
# function that perform scalar product of vectors 'x' with 'n' multiplied by 't'
def f(x,t):
    n = np.arange(x.shape[0]) # create 'n' vector as array of numbers from 0 to length of 'x'
    return np.sum(x*n)*t

x = np.random.randn(10) # take input data as array of 10 random values
t = 15 # parameter
print('f(x,t) =',f(x,t) )
# ag.grad(f,y) - provides a partial derivative of 'f' function with respect 'y' variable  
df = ag.grad(f,0) #define gradient
print('df/dx = ',df(x,t))

**Note**: autograd works with non integer arrays because you can not differentiate integer data.

There are some assumptions in previous consideration.

Firstly, in **least squares method** we assumed that we have a linear dependence from parameters but in the case of non-linear dependence we also need to use non-linear approximation.

**Problem 3.** Using [``autograd``](https://github.com/HIPS/autograd) package, try another non-linear approximation of $y(x)$, 
e.g. by [rational function](https://en.wikipedia.org/wiki/Rational_function) (see also [Padé approximant](https://en.wikipedia.org/wiki/Pad%C3%A9_approximant)). 

# Answer 3 

In [ ]:
iter = 100_000

def pade_func(x, params):
    a, b, c, d, e = params
    return (a * x**2 + b * x + c) / (d * x + e)

# Define the loss function to minimize
def loss(params, x, y):
    y_pred = pade_func(x, params)
    return np.mean((y - y_pred)**2)

params_init = np.array([1., 1., 1., 1., 1.])

grad_loss = ag.grad(loss, argnum=0)

train_loss, test_loss = np.zeros((iter,), dtype=np.float64), np.zeros((iter,), dtype=np.float64)

learning_rate = 0.1
params = params_init.copy()

optimal_params = params_init.copy()
optimal_iter = -1
min_loss = float("inf")

for i in range(iter):
    gradient = grad_loss(params, X_train, y_train)
    params -= learning_rate * gradient
    train_loss[i], test_loss[i] = loss(params, X_train, y_train), loss(params, X_test, y_test)
    if test_loss[i] < min_loss:
        optimal_params = params
        min_loss = test_loss[i]
        optimal_iter = i + 1

print("Optimal parameters:", optimal_params)
print("Optimal iteration:", optimal_iter)

In [ ]:
plt.plot(train_loss, label="train loss")
plt.plot(test_loss, label="test loss")
plt.yscale("log")
plt.xscale("log")
plt.xlabel("iteration")
plt.ylabel("MSE")
plt.legend()

In [ ]:
plt.scatter(X, Y, label='Original data')
plt.scatter(X, pade_func(X, optimal_params), label='Fitted Padé approximant')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Secondly, we assumed that noise is a random value with normal probability distribution. 
Let p and q are probability distributions of random values 'x' and 'y'. The cross-entropy of this distribution can be defined as
$H(p,q) = -\sum_{x\in X} p(x)\, \log q(x)$

**Problem 4.** Try to use [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy) as a loss function to evaluate distribution of the error term $W=Y-f(X)$. 
Compare the result with the least squares method. 
Train the method using noise with any not normal probability distribution.

For example:

a) [Student's t-distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) for noise $W$: $f_W (x) = \dfrac{\Gamma\left(\dfrac{\nu+1}{2}\right)} {\sqrt{\nu\pi}\,\Gamma\left(\dfrac{\nu}{2}\right)} \left(1+\dfrac{x^2}\nu \right)^{-(\nu+1)/2}$

b) Uniform distribution for noise $ f_W = \dfrac{1}{b-a}$ for $a<x<b$

# Answer 4

https://www.quora.com/Can-we-use-cross-entropy-to-solve-the-regression-problem

In [ ]:
y_train, y_test = np.sin(X_train) + np.random.laplace(size=X_train.shape[0])*0.05, np.sin(X_test) + np.random.laplace(size=X_test.shape[0])*0.05

In [ ]:
plt.scatter(X_train, y_train)

In [ ]:
D = 5

params_mae, params_mse = np.array([1., 1., 1., 1., 1.]), np.array([1., 1., 1., 1., 1.])

grad_mae = ag.grad(mae)
grad_mse = ag.grad(mse)



iter = 1000
learning_rate = 0.01

train_loss_mae, test_loss_mae = np.zeros((iter,), dtype=np.float64), np.zeros((iter,), dtype=np.float64)
train_loss_mse, test_loss_mse = np.zeros((iter,), dtype=np.float64), np.zeros((iter,), dtype=np.float64)


for i in range(iter):
    gradient_mse = grad_mse(params_mse, X_train, y_train)
    params_mse -= learning_rate * gradient_mse

    train_loss_mse[i] = mse(params_mse, X_train, y_train)
    test_loss_mse[i] = mse(params_mse, X_test, y_test)

learning_rate = 0.01
for i in range(iter):
    gradient_mae = grad_mae(params_mae, X_train, y_train)
    params_mae -= learning_rate * gradient_mae

    train_loss_mae[i] = mae(params_mae, X_train, y_train)
    test_loss_mae[i] = mae(params_mae, X_test, y_test)

In [ ]:
plt.plot(train_loss_mae, label='train')
plt.plot(test_loss_mae, label='test')
plt.legend()
plt.yscale("log")

In [ ]:
plt.plot(train_loss_mse, label='train')
plt.plot(test_loss_mse, label='test')
plt.legend()
plt.yscale("log")

**Quick recap**
1) Define model
2) Separate data on training and testing parts
3) Calculate gradient (for example, using autograd)
4) Optimize 
5) Check overfitting (for example cross-validation)

**Example 6** Find optimal value of parameter $\theta$ where minimum of Loss function $R(x,y,\theta)$ is achieved using autograd.

Here we will use loss function from **Example 3** defined as a mean square error.
We start from definition of a gradient and then introduce optimization function

In [ ]:
dR = ag.grad(fun=R, argnum=0)
# print( R(theta_optimal, X, Y) )
print( dR(theta_optimal, X, Y) )
print( dR(theta_lsq, X, Y) )

In [ ]:
theta0 = theta_optimal

def optimize(x,y, theta, alpha, tol=1e-6, maxiter=10000):
  for it in range(maxiter):
    loss = R(theta=theta, x=x, y=y)
    dloss = dR(theta, x, y)
    norm_dloss = np.linalg.norm(dloss.flatten())
    if it%100 == 0:
      print(f"{it}: {loss} {norm_dloss}")
    if norm_dloss<tol: return theta
    theta = theta - alpha*dloss
  return theta

In [ ]:
theta0 = optimize(x=X, y=Y, theta=theta0, alpha=5e-5)
print(theta0) 